<a href="https://colab.research.google.com/github/jalva80/NLPlaying/blob/master/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Natural Language Processing - Named Entity Recognition
Notebook de jeu avec la reconnaissance d'entités nommées.

# Librairies

Utilisation de NLTK et Spacy pour la reconnaissance d'entités nommées dans des textes bruts rédigés en français.

In [0]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag